## Step 1: Import Required Libraries 

Import the required libraries, including pandas for data handling, JSON for reading configuration files, OS for interacting with the operating system, and the Snowflake connector for connecting to Snowflake. 

In [2]:
import pandas as pd
import json
import os
from snowflake.connector import connect

## Step 2: Load Configuration 

Read the configuration details from a JSON file, such as warehouse, database, schema, user, password, and account details. 

## Step 3: Read CSV File 

Read the CSV file using pandas, specifying the delimiter as a pipe ("|"). This loads the dataset into a DataFrame for inspection. 

In [1]:
with open('config.json', 'r') as file:
    config = json.load(file)
# Define the schema, table name, and file path for a specific CSV file
schemaName = config['schema']
tblName = config['database']

file_path = 'dataset/items.csv'
df = pd.read_csv(file_path,sep="|")

# Print the first 5 rows to understand the structure
print(df.dtypes)

FileNotFoundError: [Errno 2] No such file or directory: 'Exploring_Snowflake/config.json'

## Step 4: Connect to Snowflake 

Establish a connection to Snowflake using the loaded configuration details. 

In [4]:
# Connection setup
conn = connect(
  user=config['user'],
  password=config['password'],
  account= config['account'],
)

### Step 5: Create Warehouse, Database, and Schema (If Not Exists) 

Execute SQL commands to create the warehouse, database, and schema if they don't already exist. This ensures that the proper structure is in place for the table. 

In [7]:
# after you log in, create a database, schema, and warehouse if they don’t yet exist, using the CREATE DATABASE, CREATE SCHEMA, and CREATE WAREHOUSE commands.
conn.cursor().execute(f"CREATE WAREHOUSE IF NOT EXISTS {config['warehouse']}")
conn.cursor().execute(f"CREATE DATABASE IF NOT EXISTS {config['database']}")
conn.cursor().execute(f"USE DATABASE {config['database']}")
conn.cursor().execute(f"CREATE SCHEMA IF NOT EXISTS {config['schema']}")

## Step 6: Set Context 

Use the "USE" statements to set the context to the correct warehouse, database, and schema.

In [6]:
conn.cursor().execute(f"USE WAREHOUSE {config['warehouse']}")
conn.cursor().execute(f"USE DATABASE {config['database']}")
conn.cursor().execute(f"USE SCHEMA {config['schema']}")

## Step 7: Create Table 

Define and execute the "CREATE TABLE" statement, specifying the table's schema. The IF NOT EXISTS clause ensures that the table is created only if it doesn't already exist. The schema includes data types and column names that match the CSV file's structure. 

In [8]:
# Define the CREATE TABLE statement
create_table_stmt = f"""
CREATE TABLE IF NOT EXISTS {config['testtable']} (
    pid INT,
    manufacturer INT,
    "group" STRING,
    content STRING,
    unit STRING,
    pharmForm STRING,
    genericProduct INT,
    salesIndex INT,
    category FLOAT,
    campaignIndex STRING,
    rrp FLOAT
);
"""

# Execute the CREATE TABLE statement
conn.cursor().execute(create_table_stmt)

## Step 8: Define File Format 

Create a named file format in Snowflake that matches the CSV file's structure, specifying the CSV type with a pipe ("|") as the field delimiter and skipping the header. 

In [9]:
# Define the CREATE FILE FORMAT statement
create_file_format_stmt = """
CREATE OR REPLACE FILE FORMAT my_file_format
  TYPE = 'CSV'
  FIELD_DELIMITER = '|'
  SKIP_HEADER = 1;
"""

# Execute the CREATE FILE FORMAT statement
conn.cursor().execute(create_file_format_stmt)

## Step 9: Copy Data into Table

Execute the "COPY INTO" command to import the data from the CSV file into the table using the defined file format. 

In [10]:
copy_into_stmt = """
COPY INTO testtable
FROM @%testtable
FILE_FORMAT = my_file_format;
"""
conn.cursor().execute(copy_into_stmt)

## Step 10: Put Data (Alternative Method) 

Alternatively, you can use the "PUT" command to upload the CSV file to a stage associated with the table, followed by the "COPY INTO" command to copy the data from the stage into the table. 

In [11]:
# Putting Data
conn.cursor().execute("PUT file:///Users/varshasureshbabu/Desktop/Exploring_Snowflake/Dataset/items.csv* @%testtable")
conn.cursor().execute("COPY INTO testtable")

* Note: The code snippet includes both the "COPY INTO" command directly from the CSV file (Step 9) and the "PUT" followed by "COPY INTO" commands (Step 10). Use either one of these approaches based on your specific requirements. 